In [1]:
import sys
sys.path.append('../../')
from EasyLM.models.gptj.gptj_model import (
    GPTJConfig, FlaxGPTJForCausalLMModule
)
from tqdm import tqdm, trange
import sys
from jax.experimental.pjit import pjit
from EasyLM.jax_utils import (
    JaxRNG, JaxDistributedConfig, next_rng, match_partition_rules,
    cross_entropy_loss_and_accuracy, global_norm, get_float_dtype_by_name,
    set_random_seed, average_metrics, get_weight_decay_mask,
    make_shard_and_gather_fns, with_sharding_constraint, average_metrics, get_jax_mesh
)

set_random_seed(22)

/home/asetlur/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import Dataset
import torch

def prefix_target_list(filename=None):
    """
    Load graphs and split them into prefix and target and return the list
    """
    data_list = []
    with open(filename, 'r') as f:
        lines = f.readlines()
    for line in lines:
        prefix = line.strip().split('=')[0] + '='
        target = line.strip().split('=')[1]
        target = target.split(',')[1]
        data_list.append((prefix, target))
    return data_list


class Graphs(Dataset):
    def __init__(self, tokenizer, n_samples, data_path):
        self.tokenizer = tokenizer
        self.n_samples = n_samples
        self.data_path = data_path
        self.eval_mode = False
        self.data_file = prefix_target_list(self.data_path)
        self.tokenized, self.num_prefix_tokens, self.num_target_tokens = self.tokenize(self.data_file[:n_samples])

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, idx):
        if self.eval_mode:
            # In eval mode return the entire sequence
            return self.tokenized[idx].to(self.device)

        # Create inputs
        x = self.tokenized[idx].clone()
        y = torch.cat([-torch.ones((self.num_prefix_tokens - 1, )),
                       x[self.num_prefix_tokens:].clone()])
        return x[:-1], y.long()

    def tokenize(self, data_list):
        """
        Takes a list of prefix-target pairs, tokenizes and concatenates them
        """
        out = []
        prefix_len = len(self.tokenizer.encode(data_list[0][0]))
        target_len = len(self.tokenizer.encode(data_list[0][1]))
        same_len = True

        for prefix, target in data_list:
            prefix = torch.tensor(self.tokenizer.encode(prefix))
            target = torch.tensor(self.tokenizer.encode(target))
            if not (len(prefix) == prefix_len and len(target) == target_len):
                same_len = False
            seq = torch.concatenate([prefix, target], dim=-1).long()
            out.append(seq)

        # Check if all prefixes and all targets have the same length
        if not same_len:
            print('Not all prefixes or targets have the same length!!')
        else:
            print('Equal sequence lengths!')

        return out, prefix_len, target_len

    def eval(self):
        # Switch to "eval" mode when generating sequences without teacher-forcing
        self.eval_mode = True

    def train(self):
        # Switch back to "train" mode for teacher-forcing
        self.eval_mode = False

In [3]:
from torch.utils.data import DataLoader

# LOAD TOKENIZER
from transformers import AutoTokenizer # type: ignore
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B')
tokenizer.pad_token_id = tokenizer.eos_token_id

# LOAD DATASET
data_path = 'deg_2_path_4_nodes_10'
train_path, test_path = data_path + '_train_200000.txt', data_path + '_test_20000.txt'
train_data = Graphs(tokenizer=tokenizer, n_samples=32, data_path=train_path)
test_data = Graphs(tokenizer=tokenizer, n_samples=100, data_path=test_path)
train_data.train()

# sanity check
print(train_data[0], tokenizer.decode(train_data[0][0]), tokenizer.decode(train_data[0][1][-train_data.num_target_tokens:]))

# LOAD DATALOADER
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True) 

Equal sequence lengths!
Equal sequence lengths!


2024-04-23 22:41:30.581446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-04-23 22:41:31.262694: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-04-23 22:41:31.262793: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-04-23 22:41:31.262799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use

(tensor([21, 11, 24, 91, 22, 11, 21, 91, 24, 11, 15, 91, 20, 11, 17, 91, 22, 11,
        19, 91, 19, 11, 20, 14, 22, 11, 17, 28]), tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, 19])) 6,9|7,6|9,0|5,2|7,4|4,5/7,2= 4


In [4]:
# LOAD MODEL

gpt_config = GPTJConfig()
gpt_config.update(dict(
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
))

model = FlaxGPTJForCausalLMModule(gpt_config, dtype=get_float_dtype_by_name('fp32'))

In [5]:
import math
from EasyLM.optimizers import OptimizerFactory

num_epochs = 100
warmup_ratio = 1.
learning_rate = 1e-5
weight_decay = 0.
end_lr = 1e-5
simulated_steps_per_epoch = len(train_loader)

optimizer_config = OptimizerFactory.get_default_config()
total_simulated_steps = num_epochs * simulated_steps_per_epoch
optimizer_config.adamw_optimizer.clip_gradient = 10000.
optimizer_config.adamw_optimizer.lr_decay_steps = total_simulated_steps
optimizer_config.adamw_optimizer.warmup_ratio = warmup_ratio
optimizer_config.adamw_optimizer.end_lr = end_lr
optimizer_config.adamw_optimizer.init_lr = 1e-5
optimizer_config.adamw_optimizer.lr = learning_rate
optimizer_config.adamw_optimizer.weight_decay = weight_decay

if optimizer_config.adamw_optimizer.warmup_ratio > 0:
    optimizer_config.adamw_optimizer.lr_warmup_steps = math.ceil(optimizer_config.adamw_optimizer.warmup_ratio * total_simulated_steps)

print(f"Total simulated steps: {total_simulated_steps}")
print(f"Total simulated warmup steps: {optimizer_config.adamw_optimizer.lr_warmup_steps}")
print(f"Total simulated decay steps: {optimizer_config.adamw_optimizer.lr_decay_steps}")

optimizer, optimizer_info = OptimizerFactory.get_optimizer(
    optimizer_config,
)

Total simulated steps: 100
Total simulated warmup steps: 100
Total simulated decay steps: 100


In [6]:
optimizer_config

accumulate_gradient_steps: 1
adamw_optimizer:
  b1: 0.9
  b2: 0.95
  bf16_momentum: false
  clip_gradient: 10000.0
  end_lr: 1.0e-05
  init_lr: 1.0e-05
  lr: 1.0e-05
  lr_decay_steps: 100
  lr_warmup_steps: 100
  multiply_by_parameter_scale: false
  warmup_ratio: 1.0
  weight_decay: 0.0
palm_optimizer:
  b1: 0.9
  b2: 0.99
  bf16_momentum: false
  clip_gradient: 1.0
  lr: 0.01
  lr_warmup_steps: 10000
  weight_decay: 0.0001
type: adamw

In [7]:
# running epochs

import jax
import jax.numpy as jnp
from jax.experimental.pjit import pjit
from jax.sharding import PartitionSpec as PS
from flax.training.train_state import TrainState


seq_length = train_data.num_prefix_tokens + train_data.num_target_tokens  

def create_trainstate_from_params(params):
    return TrainState.create(params=params, tx=optimizer, apply_fn=None)


def init_fn(rng):
    rng_generator = JaxRNG(rng)
    params = model.init(
        input_ids=jnp.zeros((4, seq_length), dtype=jnp.int32),
        position_ids=jnp.zeros((4, seq_length), dtype=jnp.int32),
        attention_mask=jnp.ones((4, seq_length), dtype=jnp.int32),
        rngs=rng_generator(gpt_config.rng_keys()),
    )
    return TrainState.create(params=params, tx=optimizer, apply_fn=None)


print("Initializing training state and pjitting...")
train_state_shapes = jax.eval_shape(init_fn, next_rng())
train_state_partition = match_partition_rules(
    GPTJConfig.get_partition_rules(), train_state_shapes
)

axis_dims = '-1,4,1'
mesh = get_jax_mesh(axis_dims, ('dp', 'fsdp', 'mp'))


def eval_step(train_state, rng, batch):
    rng_generator = JaxRNG(rng)
    batch = with_sharding_constraint(batch, PS(('dp', 'fsdp')))
    logits = model.apply(
        train_state.params, batch['input_tokens'], batch['attention_mask'],
        deterministic=True, rngs=rng_generator(GPTJConfig.rng_keys()),
    ).logits
    loss, accuracy = cross_entropy_loss_and_accuracy(
        logits, batch['target_tokens'], batch['loss_masks']
    )
    metrics = dict(
        eval_loss=loss,
        eval_accuracy=accuracy,
    )
    return rng_generator(), metrics

def train_step(train_state, rng, batch):
    rng_generator = JaxRNG(rng)
    batch = with_sharding_constraint(batch, PS(('dp', 'fsdp')))
    def loss_and_accuracy(params):
        logits = model.apply(
            params, batch['input_tokens'], batch['attention_mask'],
            deterministic=False, rngs=rng_generator(GPTJConfig.rng_keys()),
        ).logits
        return cross_entropy_loss_and_accuracy(
            logits, batch['target_tokens'], batch['loss_masks']
        )
    grad_fn = jax.value_and_grad(loss_and_accuracy, has_aux=True)
    (loss, accuracy), grads = grad_fn(train_state.params)
    train_state = train_state.apply_gradients(grads=grads)
    metrics = dict(
        loss=loss,
        accuracy=accuracy,
        learning_rate=optimizer_info['learning_rate_schedule'](train_state.step // optimizer_config.accumulate_gradient_steps),
        gradient_norm=global_norm(grads),
        param_norm=global_norm(train_state.params),
    )
    return train_state, rng_generator(), metrics

print("Initializing training state and pjitting...")
train_state_shapes = jax.eval_shape(init_fn, next_rng())
train_state_partition = match_partition_rules(
    GPTJConfig.get_partition_rules(), train_state_shapes
)

shard_fns, gather_fns = make_shard_and_gather_fns(
    train_state_partition, train_state_shapes
)

sharded_init_fn = pjit(
    init_fn,
    in_shardings=PS(),
    out_shardings=train_state_partition
)

sharded_create_trainstate_from_params = pjit(
    create_trainstate_from_params,
    in_shardings=(train_state_partition.params, ),
    out_shardings=train_state_partition,
    donate_argnums=(0, ),
)

sharded_train_step = pjit(
    train_step,
    in_shardings=(train_state_partition, PS(), PS()),
    out_shardings=(train_state_partition, PS(), PS()),
    donate_argnums=(0, 1),
)

sharded_eval_step = pjit(
    eval_step,
    in_shardings=(train_state_partition, PS(), PS()),
    out_shardings=(PS(), PS()),
    donate_argnums=(1,),
)

Initializing training state and pjitting...
Initializing training state and pjitting...


In [8]:
# train_state_shapes
import numpy as np


In [9]:
param_count = sum(np.prod(x.shape) for x in jax.tree_leaves(train_state_shapes.params))
print(param_count)

6050882784


/tmp/ipykernel_463255/1639233076.py:1: FutureWarning: jax.tree_leaves is deprecated, and will be removed in a future release. Use jax.tree_util.tree_leaves instead.
  param_count = sum(np.prod(x.shape) for x in jax.tree_leaves(train_state_shapes.params))


In [10]:
import time
from jax_smi import initialise_tracking
initialise_tracking()

# with mesh:
#     train_state = sharded_init_fn(next_rng())

#     start_step = int(jax.device_get(train_state.step))
#     start_epoch = start_step // simulated_steps_per_epoch
#     start_step = start_step % simulated_steps_per_epoch

#     sharded_rng = next_rng()

#     epoch_counter = trange(start_epoch, num_epochs, ncols=0, position=0)
#     step_counter = trange(start_step, simulated_steps_per_epoch, ncols=0, position=1)
    
    # overall_step = 0
    # for epoch in epoch_counter:
    #     for step, batch in tqdm(zip(step_counter, train_loader), total=simulated_steps_per_epoch):
    #         x, y = batch
    #         x = x.numpy()
    #         y = y.numpy()
            # if isinstance(batch, (list, tuple)):
            #     batch = {
            #         'input_tokens': x,
            #         'target_tokens': y,
            #         'loss_masks': (y != -1).astype(jnp.int32),
            #         'attention_mask': (y != -1).astype(jnp.int32)
            #     }
            # # just measuring the train step time.
            # start_time = time.time()
            # train_state, sharded_rng, metrics = sharded_train_step(
            #     train_state, sharded_rng, batch
            # )
            # step_time = time.time() - start_time
            # overall_step += 1

    #         if step % FLAGS.log_freq == 0:
    #             if FLAGS.eval_steps > 0:
    #                 eval_metric_list = []
    #                 for batch in eval_dataset:
    #                     if isinstance(batch, (list, tuple)):
    #                         batch = {
    #                             'tokens': batch[0],
    #                             'loss_masks': batch[1],
    #                         }
    #                     sharded_rng, eval_metrics = sharded_eval_step(
    #                         train_state, sharded_rng, batch
    #                     )
    #                     eval_metric_list.append(eval_metrics)
    #                 metrics.update(average_metrics(eval_metric_list))
    #             log_metrics = {
    #                 "train/step": overall_step,
    #                 "train/samples_seen": overall_step * real_batch_size,
    #                 "train/step_time": step_time,
    #                 "train/epoch": overall_step / steps_per_epoch,
    #             }
    #             log_metrics = jax.device_get(log_metrics)
    #             log_metrics.update(metrics)
    #             log_metrics = {k: float(v) for k, v in log_metrics.items()}
    #             tqdm.write("\n" + pprint.pformat(log_metrics) + "\n")


In [11]:
import pprint

with mesh:
    train_state = sharded_init_fn(next_rng())
    start_step = int(jax.device_get(train_state.step))
    start_epoch = start_step // simulated_steps_per_epoch
    start_step = start_step % simulated_steps_per_epoch
    sharded_rng = next_rng()
    # epoch_counter = trange(start_epoch, num_epochs, ncols=0, position=0)
    step_counter = trange(start_step, simulated_steps_per_epoch, ncols=0, position=1)
    overall_step = 0
    # num_epochs = 100
    for epoch in range(num_epochs):
        for step, batch in tqdm(zip(step_counter, train_loader), total=simulated_steps_per_epoch):
            overall_step += 1
            x, y = batch
            x = x.numpy()
            y = y.numpy()
            batch = {
                'input_tokens': x,
                'target_tokens': y,
                'loss_masks': (y != -1).astype(jnp.int32),
                'attention_mask': np.ones_like(x).astype(jnp.int32)
            }
            # just measuring the train step time.
            start_time = time.time()
            train_state, sharded_rng, metrics = sharded_train_step(
                train_state, sharded_rng, batch
            )
            step_time = time.time() - start_time

            # if log_fr > 0:
            #         eval_metric_list = []
            #         for batch in eval_dataset:
            #             if isinstance(batch, (list, tuple)):
            #                 batch = {
            #                     'tokens': batch[0],
            #                     'loss_masks': batch[1],
            #                 }
            #             sharded_rng, eval_metrics = sharded_eval_step(
            #                 train_state, sharded_rng, batch
            #             )
            #             eval_metric_list.append(eval_metrics)
                    # metrics.update(average_metrics(eval_metric_list))
            if overall_step % 1 == 0:
                log_metrics = {
                    "train/step": overall_step,
                    "train/samples_seen": overall_step * 64,
                    "train/step_time": step_time,
                    "train/epoch": overall_step / simulated_steps_per_epoch,
                }
                log_metrics = jax.device_get(log_metrics)
                log_metrics.update(metrics)
                log_metrics = {k: float(v) for k, v in log_metrics.items()}
                tqdm.write("\n" + pprint.pformat(log_metrics) + "\n")

  0%|          | 0/1 [00:00<?, ?it/s]
100% 1/1 [02:33<00:00, 153.26s/it] 153.26s/it]
100%|██████████| 1/1 [02:33<00:00, 153.26s/it]



{'accuracy': 0.0,
 'gradient_norm': 70.19483947753906,
 'learning_rate': 9.999999747378752e-06,
 'loss': 11.480037689208984,
 'param_norm': 1133.121337890625,
 'train/epoch': 1.0,
 'train/samples_seen': 64.0,
 'train/step': 1.0,
 'train/step_time': 152.94790744781494}



100%|██████████| 1/1 [00:00<00:00,  2.56it/s]



{'accuracy': 0.3125,
 'gradient_norm': 81.10279846191406,
 'learning_rate': 9.999999747378752e-06,
 'loss': 4.586386680603027,
 'param_norm': 1133.12158203125,
 'train/epoch': 2.0,
 'train/samples_seen': 128.0,
 'train/step': 2.0,
 'train/step_time': 0.08667612075805664}



100%|██████████| 1/1 [00:00<00:00,  2.75it/s]


...


{'accuracy': 1.0,
 'gradient_norm': 0.0013074430171400309,
 'learning_rate': 9.999999747378752e-06,
 'loss': 2.0287658117013052e-05,
 'param_norm': 1133.1356201171875,
 'train/epoch': 98.0,
 'train/samples_seen': 6272.0,
 'train/step': 98.0,
 'train/step_time': 0.060712337493896484}



100%|██████████| 1/1 [00:00<00:00,  2.74it/s]



{'accuracy': 1.0,
 'gradient_norm': 0.0012514239642769098,
 'learning_rate': 9.999999747378752e-06,
 'loss': 1.9717765098903328e-05,
 'param_norm': 1133.1357421875,
 'train/epoch': 99.0,
 'train/samples_seen': 6336.0,
 'train/step': 99.0,
 'train/step_time': 0.06013369560241699}



100%|██████████| 1/1 [00:00<00:00,  2.75it/s]


{'accuracy': 1.0,
 'gradient_norm': 0.0011923854472115636,
 'learning_rate': 9.999999747378752e-06,
 'loss': 1.9166473066434264e-05,
 'param_norm': 1133.1357421875,
 'train/epoch': 100.0,
 'train/samples_seen': 6400.0,
 'train/step': 100.0,
 'train/step_time': 0.06075477600097656}



In [12]:
sys.argv = """
    --mesh_dim='-1,64,1' \
    --dtype='fp32' \
    --total_steps=250000 \
    --log_freq=50 \
    --save_model_freq=0 \
    --save_milestone_freq=2500 \
    --load_llama_config='7b' \
    --update_llama_config='' \
    --load_dataset_state='' \
    --load_checkpoint='' \
    --tokenizer.vocab_file='./llama2-tokenizer.model' \
    --optimizer.type='adamw' \
    --optimizer.adamw_optimizer.weight_decay=0.1 \
    --optimizer.adamw_optimizer.lr=3e-4 \
    --optimizer.adamw_optimizer.end_lr=3e-5 \
    --optimizer.adamw_optimizer.lr_warmup_steps=2000 \
    --optimizer.adamw_optimizer.lr_decay_steps=250000 \
    --train_dataset.type='json' \
    --train_dataset.text_processor.fields='text' \
    --train_dataset.json_dataset.path='/path/to/shuffled/redpajama/dataset' \
    --train_dataset.json_dataset.seq_length=2048 \
    --train_dataset.json_dataset.batch_size=2048 \
    --train_dataset.json_dataset.tokenizer_processes=16 \
    --checkpointer.save_optimizer_state=True \
    --logger.online=True \
    --logger.prefix='EasyLM' \
    --logger.project="open_llama_7b" \
    --logger.output_dir="/path/to/checkpoint/dir" \
    --logger.wandb_dir="$HOME/experiment_output/open_llama_7b" \
""".split("=")